In [1]:
import pandas as pd
import geopandas
from geopy.geocoders import Nominatim

/usr/local/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [2]:
#load in latest dspt snapshot
dspt_snapshot = pd.read_csv('data/DSPT Snapshots/22_23/DSPT search results 17_10_2023 08_55_43.csv')
dspt_snapshot.head()

,Code,Organisation Name,Status,Date Of Publication,Primary Sector,"Integrated Care Board (where available, from ODS)"
0,00L,NHS NORTH EAST AND NORTH CUMBRIA ICB - 00L (CCG),21/22 Standards Met,30/06/2022,CCG / Integrated Care Board (ICB),NHS NORTH EAST AND NORTH CUMBRIA INTEGRATED CA...
1,00N,NHS NORTH EAST AND NORTH CUMBRIA ICB - 00N (CCG),21/22 Standards Met,30/06/2022,CCG / Integrated Care Board (ICB),NHS NORTH EAST AND NORTH CUMBRIA INTEGRATED CA...
2,00P,NHS NORTH EAST AND NORTH CUMBRIA ICB - 00P (CCG),21/22 Standards Met,28/06/2022,CCG / Integrated Care Board (ICB),NHS NORTH EAST AND NORTH CUMBRIA INTEGRATED CA...
3,00Q,NHS LANCASHIRE AND SOUTH CUMBRIA ICB - 00Q (CCG),21/22 Standards Met,22/06/2022,CCG / Integrated Care Board (ICB),NHS LANCASHIRE AND SOUTH CUMBRIA INTEGRATED CA...
4,00R,NHS LANCASHIRE AND SOUTH CUMBRIA ICB - 00R (CCG),21/22 Standards Met,29/06/2022,CCG / Integrated Care Board (ICB),NHS LANCASHIRE AND SOUTH CUMBRIA INTEGRATED CA...


In [3]:
#load in auxillary trust data (etr and ect and merge)
etr = pd.read_csv('data/aux/22_23/etr.csv', header=None)
ect = pd.read_csv('data/aux/22_23/ect.csv', header=None)
trusts_combined = pd.concat([etr, ect])[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]
trusts_combined

,0,1,2,3,4,5,6,7,8,9
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,Y62,QOP,COBBETT HOUSE,OXFORD ROAD,NaN,MANCHESTER,NaN,M13 9WL
1,R0B,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,Y63,QHM,SUNDERLAND ROYAL HOSPITAL,KAYLL ROAD,NaN,SUNDERLAND,NaN,SR4 7TP
2,R0D,UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST,Y58,QVV,MANAGEMENT OFFICES,POOLE HOSPITAL,LONGFLEET ROAD,POOLE,NaN,BH15 2JB
3,R1A,HEREFORDSHIRE AND WORCESTERSHIRE HEALTH AND CA...,Y60,QGH,UNIT 2 KINGS COURT,CHARLES HASTINGS WAY,NaN,WORCESTER,NaN,WR5 1JR
4,R1C,SOLENT NHS TRUST,Y59,QRL,SOLENT NHS TRUST HEADQUARTERS,HIGHPOINT VENUE,BURSLEDON ROAD,SOUTHAMPTON,NaN,SO19 8BR
...,...,...,...,...,...,...,...,...,...,...
210,RYY,KENT COMMUNITY HEALTH NHS FOUNDATION TRUST,Y59,QKS,TRINITY HOUSE,110-120 EUREKA PARK,EUREKA BUSINESS PARK,ASHFORD,KENT,TN25 4AZ
0,TAD,BRADFORD DISTRICT CARE NHS FOUNDATION TRUST,Y63,QWO,NEW MILL,VICTORIA ROAD,SALTAIRE,SHIPLEY,NaN,BD18 3LD
1,TAF,CAMDEN AND ISLINGTON NHS FOUNDATION TRUST,Y56,QMJ,ST PANCRAS HOSPITAL,4 ST PANCRAS WAY,NaN,LONDON,NaN,NW1 0PE
2,TAH,SHEFFIELD HEALTH & SOCIAL CARE NHS FOUNDATION ...,Y63,QF7,CENTRE COURT,ATLAS WAY,NaN,SHEFFIELD,NaN,S4 7QQ


In [5]:
trusts_combined.columns = ['Code', 'Organisation Name', 'National Grouping', 'High Level Health Geography', 'Address Line 1', 'Address Line 2', 'Address Line 3', 'Address Line 4', 'Address Line 5', 'Post Code']
trusts_combined = trusts_combined.reset_index(drop=True)
trusts_combined

,Code,Organisation Name,National Grouping,High Level Health Geography,Address Line 1,Address Line 2,Address Line 3,Address Line 4,Address Line 5,Post Code
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,Y62,QOP,COBBETT HOUSE,OXFORD ROAD,NaN,MANCHESTER,NaN,M13 9WL
1,R0B,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,Y63,QHM,SUNDERLAND ROYAL HOSPITAL,KAYLL ROAD,NaN,SUNDERLAND,NaN,SR4 7TP
2,R0D,UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST,Y58,QVV,MANAGEMENT OFFICES,POOLE HOSPITAL,LONGFLEET ROAD,POOLE,NaN,BH15 2JB
3,R1A,HEREFORDSHIRE AND WORCESTERSHIRE HEALTH AND CA...,Y60,QGH,UNIT 2 KINGS COURT,CHARLES HASTINGS WAY,NaN,WORCESTER,NaN,WR5 1JR
4,R1C,SOLENT NHS TRUST,Y59,QRL,SOLENT NHS TRUST HEADQUARTERS,HIGHPOINT VENUE,BURSLEDON ROAD,SOUTHAMPTON,NaN,SO19 8BR
...,...,...,...,...,...,...,...,...,...,...
210,RYY,KENT COMMUNITY HEALTH NHS FOUNDATION TRUST,Y59,QKS,TRINITY HOUSE,110-120 EUREKA PARK,EUREKA BUSINESS PARK,ASHFORD,KENT,TN25 4AZ
211,TAD,BRADFORD DISTRICT CARE NHS FOUNDATION TRUST,Y63,QWO,NEW MILL,VICTORIA ROAD,SALTAIRE,SHIPLEY,NaN,BD18 3LD
212,TAF,CAMDEN AND ISLINGTON NHS FOUNDATION TRUST,Y56,QMJ,ST PANCRAS HOSPITAL,4 ST PANCRAS WAY,NaN,LONDON,NaN,NW1 0PE
213,TAH,SHEFFIELD HEALTH & SOCIAL CARE NHS FOUNDATION ...,Y63,QF7,CENTRE COURT,ATLAS WAY,NaN,SHEFFIELD,NaN,S4 7QQ


In [6]:
locator = Nominatim(user_agent = 'myGeocoder')

In [ ]:
lat = []
long = []
for i in list(trusts_combined['Post Code']):
    lat.append(locator.geocode(i).latitude)
    long.append(locator.geocode(i).longitude)

    

In [77]:
trusts_combined = trusts_combined[['Code', 'Organisation Name', 'National Grouping', 'High Level Health Geography']]
trusts_combined['lat'] = lat
trusts_combined['long'] = long

/var/folders/kt/92wnzvcj6qq_r6j3j19yp5lw0000gp/T/ipykernel_16494/1499399131.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trusts_combined['lat'] = lat
/var/folders/kt/92wnzvcj6qq_r6j3j19yp5lw0000gp/T/ipykernel_16494/1499399131.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trusts_combined['long'] = long


In [80]:
trusts_combined

,Code,Organisation Name,National Grouping,High Level Health Geography,lat,long
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,Y62,QOP,53.462440,-2.227710
1,R0B,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,Y63,QHM,54.902770,-1.408705
2,R0D,UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST,Y58,QVV,50.721389,-1.972792
3,R1A,HEREFORDSHIRE AND WORCESTERSHIRE HEALTH AND CA...,Y60,QGH,52.192162,-2.179038
4,R1C,SOLENT NHS TRUST,Y59,QRL,50.901607,-1.329799
...,...,...,...,...,...,...
210,RYY,KENT COMMUNITY HEALTH NHS FOUNDATION TRUST,Y59,QKS,51.171653,0.870975
211,TAD,BRADFORD DISTRICT CARE NHS FOUNDATION TRUST,Y63,QWO,53.839670,-1.788610
212,TAF,CAMDEN AND ISLINGTON NHS FOUNDATION TRUST,Y56,QMJ,51.535866,-0.131679
213,TAH,SHEFFIELD HEALTH & SOCIAL CARE NHS FOUNDATION ...,Y63,QF7,53.399947,-1.440936


In [79]:
trusts_dspt = trusts_combined.merge(dspt_snapshot, on = 'Code').reset_index(drop = True)
trusts_dspt


,Code,Organisation Name_x,National Grouping,High Level Health Geography,lat,long,Organisation Name_y,Status,Date Of Publication,Primary Sector,"Integrated Care Board (where available, from ODS)"
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,Y62,QOP,53.462440,-2.227710,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,22/23 Standards Met,30/06/2023,NHS Trust,NHS GREATER MANCHESTER INTEGRATED CARE BOARD
1,R0B,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,Y63,QHM,54.902770,-1.408705,SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION T...,22/23 Standards Exceeded,15/06/2023,NHS Trust,NHS NORTH EAST AND NORTH CUMBRIA INTEGRATED CA...
2,R0D,UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST,Y58,QVV,50.721389,-1.972792,UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST,22/23 Approaching Standards,30/06/2023,NHS Trust,NHS DORSET INTEGRATED CARE BOARD
3,R1A,HEREFORDSHIRE AND WORCESTERSHIRE HEALTH AND CA...,Y60,QGH,52.192162,-2.179038,Herefordshire and Worcestershire Health and Ca...,22/23 Standards Exceeded,05/06/2023,NHS Trust,NHS HEREFORDSHIRE AND WORCESTERSHIRE INTEGRATE...
4,R1C,SOLENT NHS TRUST,Y59,QRL,50.901607,-1.329799,Solent NHS Trust,22/23 Standards Met,30/06/2023,NHS Trust,NHS HAMPSHIRE AND ISLE OF WIGHT INTEGRATED CAR...
...,...,...,...,...,...,...,...,...,...,...,...
208,RYY,KENT COMMUNITY HEALTH NHS FOUNDATION TRUST,Y59,QKS,51.171653,0.870975,Kent Community Health NHS Foundation Trust,22/23 Standards Met,27/06/2023,NHS Trust,NHS KENT AND MEDWAY INTEGRATED CARE BOARD
209,TAD,BRADFORD DISTRICT CARE NHS FOUNDATION TRUST,Y63,QWO,53.839670,-1.788610,BRADFORD DISTRICT CARE NHS FOUNDATION TRUST,22/23 Standards Met,29/06/2023,NHS Trust,NHS WEST YORKSHIRE INTEGRATED CARE BOARD
210,TAF,CAMDEN AND ISLINGTON NHS FOUNDATION TRUST,Y56,QMJ,51.535866,-0.131679,CAMDEN AND ISLINGTON NHS FOUNDATION TRUST,22/23 Approaching Standards,29/06/2023,NHS Trust,NHS NORTH CENTRAL LONDON INTEGRATED CARE BOARD
211,TAH,SHEFFIELD HEALTH & SOCIAL CARE NHS FOUNDATION ...,Y63,QF7,53.399947,-1.440936,Sheffield Health & Social Care NHS Foundation ...,22/23 Approaching Standards,30/06/2023,NHS Trust,NHS SOUTH YORKSHIRE INTEGRATED CARE BOARD


In [4]:
#check which Trusts are not in the dataframe
#set(trusts_combined['Code']) - set(trusts_dspt['Code'])

In [84]:
trusts_combined.loc[trusts_combined['Code'] == 'RVY']

,Code,Organisation Name,National Grouping,High Level Health Geography,lat,long
146,RVY,SOUTHPORT AND ORMSKIRK HOSPITAL NHS TRUST,Y62,QYG,53.63355,-2.97882


In [85]:
trusts_combined.loc[trusts_combined['Code'] == 'RYT']

,Code,Organisation Name,National Grouping,High Level Health Geography,lat,long
206,RYT,PUBLIC HEALTH WALES NHS TRUST,W00,Q99,51.476792,-3.169914


In [86]:
trusts_dspt.to_csv('data/DSPT Snapshots/dspt_trusts_snapshot_17_10_23.csv')

In [91]:
lookup_table = pd.read_csv('data/aux/22_23/Sub_ICB_Locations_to_Integrated_Care_Boards_to_NHS_England_(Region)_(April_2023)_Lookup_in_England (1).csv')
icb_table = lookup_table.drop_duplicates('ICB23CDH').reset_index(drop=True)[['ICB23CD', 'ICB23CDH', 'ICB23NM']]
icb_table

,ICB23CD,ICB23CDH,ICB23NM
0,E54000027,QRV,NHS North West London Integrated Care Board
1,E54000028,QMJ,NHS North Central London Integrated Care Board
2,E54000029,QMF,NHS North East London Integrated Care Board
3,E54000030,QKK,NHS South East London Integrated Care Board
4,E54000031,QWE,NHS South West London Integrated Care Board
5,E54000032,QKS,NHS Kent and Medway Integrated Care Board
6,E54000034,QNQ,NHS Frimley Integrated Care Board
7,E54000042,QRL,NHS Hampshire and Isle of Wight Integrated Car...
8,E54000044,QU9,"NHS Buckinghamshire, Oxfordshire and Berkshire..."
9,E54000063,QXU,NHS Surrey Heartlands Integrated Care Board


In [92]:
icb_dspt = icb_table.merge(dspt_snapshot, left_on = 'ICB23CDH', right_on = 'Code')
icb_dspt

,ICB23CD,ICB23CDH,ICB23NM,Code,Organisation Name,Status,Date Of Publication,Primary Sector,"Integrated Care Board (where available, from ODS)"
0,E54000027,QRV,NHS North West London Integrated Care Board,QRV,NHS NORTH WEST LONDON INTEGRATED CARE BOARD,22/23 Standards Exceeded,26/06/2023,CCG / Integrated Care Board (ICB),NaN
1,E54000028,QMJ,NHS North Central London Integrated Care Board,QMJ,NHS NORTH CENTRAL LONDON INTEGRATED CARE BOARD,22/23 Standards Met,30/06/2023,CCG / Integrated Care Board (ICB),NaN
2,E54000029,QMF,NHS North East London Integrated Care Board,QMF,NHS NORTH EAST LONDON INTEGRATED CARE BOARD,22/23 Standards Exceeded,30/06/2023,CCG / Integrated Care Board (ICB),NaN
3,E54000030,QKK,NHS South East London Integrated Care Board,QKK,NHS SOUTH EAST LONDON INTEGRATED CARE BOARD,22/23 Standards Exceeded,30/06/2023,CCG / Integrated Care Board (ICB),NaN
4,E54000031,QWE,NHS South West London Integrated Care Board,QWE,NHS SOUTH WEST LONDON INTEGRATED CARE BOARD,22/23 Standards Met,27/06/2023,CCG / Integrated Care Board (ICB),NaN
5,E54000032,QKS,NHS Kent and Medway Integrated Care Board,QKS,NHS KENT AND MEDWAY INTEGRATED CARE BOARD,22/23 Standards Met,18/07/2023,CCG / Integrated Care Board (ICB),NaN
6,E54000034,QNQ,NHS Frimley Integrated Care Board,QNQ,NHS FRIMLEY INTEGRATED CARE BOARD,22/23 Standards Met,30/06/2023,CCG / Integrated Care Board (ICB),NaN
7,E54000042,QRL,NHS Hampshire and Isle of Wight Integrated Car...,QRL,NHS HAMPSHIRE AND ISLE OF WIGHT INTEGRATED CAR...,22/23 Standards Exceeded,30/06/2023,CCG / Integrated Care Board (ICB),NaN
8,E54000044,QU9,"NHS Buckinghamshire, Oxfordshire and Berkshire...",QU9,"NHS BUCKINGHAMSHIRE, OXFORDSHIRE AND BERKSHIRE...",22/23 Standards Exceeded,27/06/2023,CCG / Integrated Care Board (ICB),NaN
9,E54000063,QXU,NHS Surrey Heartlands Integrated Care Board,QXU,NHS SURREY HEARTLANDS INTEGRATED CARE BOARD,22/23 Standards Exceeded,29/06/2023,CCG / Integrated Care Board (ICB),NaN


In [94]:
icb_dspt.to_csv('data/Dspt Snapshots/ICB_dspt_snapshot_17_10_23.csv')